In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime


Matplotlib is building the font cache; this may take a moment.


<class 'ModuleNotFoundError'>: No module named 'seaborn'

In [26]:
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")  
df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [27]:
print(df.isnull().sum())
df.fillna(df.select_dtypes(include=['number']).median(), inplace=True)
 


customerID          0
,gender              0
,SeniorCitizen       0
,Partner             0
,Dependents          0
,tenure              0
,PhoneService        0
,MultipleLines       0
,InternetService     0
,OnlineSecurity      0
,OnlineBackup        0
,DeviceProtection    0
,TechSupport         0
,StreamingTV         0
,StreamingMovies     0
,Contract            0
,PaperlessBilling    0
,PaymentMethod       0
,MonthlyCharges      0
,TotalCharges        0
,Churn               0
,dtype: int64


In [28]:
# Standardize column names (remove spaces, lowercase)
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Display column names to verify changes
print(df.columns.tolist())




['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents', 'tenure', 'phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod', 'monthlycharges', 'totalcharges', 'churn']


In [4]:
# Fill missing categorical values with the most frequent category (mode)
for col in df.select_dtypes(include=['object']).columns:
    df[col].fillna(df[col].mode()[0], inplace=True)

# Check if missing values remain
print("Missing values after handling:\n", df.isnull().sum())


<class 'NameError'>: name 'df' is not defined

In [30]:
# Convert to datetime safely
date_cols = ["tenure"]  # Modify if actual date columns exist

for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Create a new feature: Long-term customer (1 if tenure > 12 months, else 0)
df["long_term_customer"] = (df["tenure"] > 12).astype(int)


In [31]:
# Convert Yes/No columns to binary (1/0)
yes_no_columns = ["churn", "partner", "dependents", "phone_service", "paperless_billing"]
for col in yes_no_columns:
    if col in df.columns:
        df[col] = df[col].map({"Yes": 1, "No": 0})

# One-hot encode categorical variables
df = pd.get_dummies(df, drop_first=True)


In [2]:
plt.figure(figsize=(12, 6))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Feature Correlation")
plt.show()


<Figure size 1200x600 with 0 Axes>

<class 'NameError'>: name 'sns' is not defined

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Define Features and Target
X = df.drop(columns=["customerid", "churn"])  # Exclude ID and target column
y = df["churn"]

# Split into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


<class 'NameError'>: name 'df' is not defined

In [ ]:
# Train model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)


In [ ]:
# Train model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)


In [ ]:
import pickle

# Save Model
with open("churn_model.pkl", "wb") as file:
    pickle.dump(model, file)

# Load and Predict with Model
with open("churn_model.pkl", "rb") as file:
    loaded_model = pickle.load(file)

# Predict on New Data
new_data = X_test.iloc[0:1]
prediction = loaded_model.predict(new_data)
print("Predicted Churn:", prediction)
